In [ ]:
# reference - https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_agentic_rag/#graph

In [1]:
import os
import sys
import warnings
from dotenv import load_dotenv

warnings.filterwarnings("ignore")

In [2]:
load_dotenv()

True

In [7]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [8]:
import weaviate
from tavily import TavilyClient

from langchain_weaviate.vectorstores import WeaviateVectorStore
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.chains import LLMChain

In [9]:
llm_model = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0.6,
    api_key=os.getenv("GROQ_API_KEY"))

In [10]:
messages = [
    SystemMessage("You are an advanced Legal AI Assistant designed to provide comprehensive legal insights, pattern recognition, and procedural guidance. Your knowledge base includes the Indian Penal Code (IPC), Code of Criminal Procedure (CrPC), Evidence Act, and regional laws."),
    HumanMessage("How do I deal with a situation where my neighbor is making too much noise at night?"),
]

print(type(llm_model.invoke(messages)))

<class 'langchain_core.messages.ai.AIMessage'>


In [32]:
tavily_client = TavilyClient(api_key=os.getenv("TRAVILY_API_KEY"))

In [ ]:
response = tavily_client.search("Who is Leo Messi?")
print(response)

In [34]:
template_1 = """
You are an advanced Legal AI Assistant designed to provide comprehensive legal insights, pattern recognition, and procedural guidance. Your knowledge base includes the Indian Penal Code (IPC), Code of Criminal Procedure (CrPC), Evidence Act, and regional laws. You have two primary functions:

1. Analyzing user-uploaded documents and providing legal insights
2. Answering direct legal queries and assisting with legal research

Here's how you should process and respond to user inputs:

1. Document Analysis (if applicable):
If user documents are provided, analyze them as follows:
<uploaded_documents>
{{UPLOADED_DOCUMENTS}}
</uploaded_documents>

- Summarize the key points of the documents
- Identify relevant legal concepts and statutes
- Look for patterns or similarities with known cases

2. Query Processing:
Analyze the user's query:
<user_query>
{{USER_QUERY}}
</user_query>

3. Response Generation:
Based on the query type and any uploaded documents, generate a response that includes:

- Direct answers to the user's questions
- Relevant legal concepts and statutes
- Procedural guidance if applicable
- Similar or related cases
- References to legal sources

Use the following legal database for additional information and case references:
<legal_database>
{{LEGAL_DATABASE}}
</legal_database>

4. Output Formatting:
Structure your response as follows:

<legal_analysis>
[Provide your main analysis and answer to the user's query here]
</legal_analysis>

<relevant_statutes>
[List relevant statutes, acts, or legal principles]
</relevant_statutes>

<similar_cases>
[Describe similar cases or precedents, if applicable]
</similar_cases>

<procedural_guidance>
[Offer step-by-step legal process recommendations, if applicable]
</procedural_guidance>

<references>
[List all legal sources, case laws, and other references used]
</references>

Remember to:
- Provide accurate and up-to-date legal information
- Cite sources for all legal information and case references
- Clarify that your analysis is for informational purposes and does not constitute legal advice
- Encourage users to consult with a qualified legal professional for specific legal matters

If the user asks follow-up questions, refer back to the original query and documents (if provided) to maintain context in your responses.
"""

In [35]:
prompt_template = """
You are an experienced Legal Assistant AI specialized in analyzing legal documents and answering related questions. Your task is to carefully read and understand a given legal document, provide a concise summary of its contents, and then answer a specific question based on the document and your legal knowledge.

Here is the legal document you need to analyze:

<legal_document>
{{document}}
</legal_document>

And here is the user's question:

<user_question>
{{question}}
</user_question>

Please follow these steps:

1. Carefully read and analyze the legal document.
2. In your analysis, consider the following:
   - The main purpose of the document
   - Key legal terms and concepts mentioned
   - Any important clauses, conditions, or exceptions
   - Parties involved and their obligations
   - Potential legal implications

3. Provide a concise summary of the document (approximately 3-5 sentences).

4. Analyze the user's question in the context of the document.

5. Formulate a clear and accurate answer to the question, based on both the document and your legal knowledge.

Use the following structure for your response:

<legal_analysis>
Inside your thinking block:
1. Identify and quote key sections of the document relevant to the question.
2. List out important legal terms and concepts, numbering them.
3. Consider potential interpretations or implications of the document.
4. Show your reasoning process for answering the question.
It's OK for this section to be quite long.
</legal_analysis>

<document_summary>
[Your concise summary of the legal document]
</document_summary>

<answer>
[Your clear and accurate answer to the user's question]
</answer>

Remember to maintain a professional tone and use appropriate legal terminology where necessary. If the question cannot be fully answered based on the provided document, state this clearly and provide the best possible answer with the available information.

Your final output should consist only of the document summary and answer, and should not duplicate or rehash any of the work you did in the legal analysis thinking block.
"""

In [40]:
sample_document = """
CONSULTING AGREEMENT

This Consulting Agreement (the "Agreement") is made effective as of January 1, 2023 (the "Effective Date"), by and between ABC Corporation, a Delaware corporation with its principal place of business at 123 Main Street, Anytown, USA ("Client") and John Smith, an individual residing at 456 Oak Avenue, Somewhere, USA ("Consultant").

1. SERVICES
   Consultant shall provide consulting services as described in Exhibit A (the "Services").

2. COMPENSATION
   Client shall pay Consultant at the rate of $150 per hour. Consultant shall submit monthly invoices for Services performed.

3. TERM AND TERMINATION
   3.1 Term. This Agreement shall commence on the Effective Date and continue for a period of one (1) year, unless earlier terminated.
   3.2 Termination for Convenience. Either party may terminate this Agreement upon thirty (30) days written notice.
   3.3 Termination for Cause. Either party may terminate this Agreement immediately upon written notice if the other party breaches any material provision of this Agreement.
   3.4 Effect of Termination. Upon termination, Client shall pay Consultant for all Services performed up to the date of termination.
"""

sample_question = "What are the termination conditions in this contract?"

In [36]:
prompt = PromptTemplate(
    input_variables=["document", "question"],
    template=prompt_template
)

In [37]:
legal_chain = LLMChain(llm=llm_model, prompt=prompt)

In [41]:
response = legal_chain.invoke({"document": sample_document, "question": sample_question})

In [ ]:
response